In [1]:
from pond.tensor import NativeTensor, PrivateEncodedTensor, PublicEncodedTensor
from pond.nn import Dense, Sigmoid, Reveal, Diff, Softmax, CrossEntropy, Sequential

In [2]:
import numpy as np
from datetime import datetime

In [8]:
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [9]:
def preprocess_data(dataset):
    
    (x_train, y_train), (x_test, y_test) = dataset
    
    # NOTE: this is the shape used by Tensorflow; other backends may differ
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test  = x_test.reshape(x_test.shape[0], 28, 28, 1)
    
    x_train  = x_train.astype('float32')
    x_test   = x_test.astype('float32')
    x_train /= 255
    x_test  /= 255

    y_train = to_categorical(y_train, 5)
    y_test  = to_categorical(y_test, 5)
    
    return (x_train, y_train), (x_test, y_test)

def load_data():
    
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train_public = x_train[y_train < 5]
    y_train_public = y_train[y_train < 5]
    x_test_public  = x_test[y_test < 5]
    y_test_public  = y_test[y_test < 5]
    public_dataset = (x_train_public, y_train_public), (x_test_public, y_test_public)

    x_train_private = x_train[y_train >= 5]
    y_train_private = y_train[y_train >= 5] - 5
    x_test_private  = x_test[y_test >= 5]
    y_test_private  = y_test[y_test >= 5] - 5
    private_dataset = (x_train_private, y_train_private), (x_test_private, y_test_private)
    
    return preprocess_data(public_dataset), preprocess_data(private_dataset)

## Pre-train on public data

In [10]:
public_dataset, private_dataset = load_data()

feature_layers = [
    keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    keras.layers.Activation('sigmoid'),
    keras.layers.Conv2D(32, (3, 3), padding='same'),
    keras.layers.Activation('sigmoid'),
    keras.layers.AveragePooling2D(pool_size=(2,2)),
    keras.layers.Dropout(.25),
    keras.layers.Flatten()
]

classification_layers = [
    keras.layers.Dense(128),
    keras.layers.Activation('sigmoid'),
    keras.layers.Dropout(.50),
    keras.layers.Dense(5),
    keras.layers.Activation('softmax')
]

model = keras.models.Sequential(feature_layers + classification_layers)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

(x_train, y_train), (x_test, y_test) = public_dataset

model.fit(
    x_train, y_train,
    epochs=1,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 30596 samples, validate on 5139 samples
Epoch 1/1
30596/30596 [==============================] - 156s - loss: 0.6478 - acc: 0.7480 - val_loss: 0.1370 - val_acc: 0.9601


## Extract features from private data (unencrypted)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
__________

In [12]:
flatten_layer = model.get_layer(index=7)
assert flatten_layer.name.startswith('flatten_')

extractor = keras.models.Model(
    inputs=model.input, 
    outputs=flatten_layer.output
)

In [41]:
(x_train_images, y_train), (x_test_images, y_test) = private_dataset

x_train_images = x_train_images[:15000]
y_train        = y_train[:15000]
x_test_images  = x_test_images[:15000]
y_test         = y_test[:15000]

x_train_features = extractor.predict(x_train_images)
x_test_features  = extractor.predict(x_test_images)

## Fine-tune

In [42]:
classifier = Sequential([
    Dense(128, 6272),
    Sigmoid(),
    # Dropout(.5),
    Dense(5, 128),
    Reveal(),
    Softmax()
])

### ... using NativeTensor

In [43]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    NativeTensor(x_train_features), 
    NativeTensor(y_train), 
    loss=CrossEntropy(), 
    epochs=3
)
stop = datetime.now()
print("Elapsed: ", stop - start)

predicted_classes = classifier.predict(NativeTensor(x_train_features)).reveal().argmax(axis=1)
correct_classes   = NativeTensor(y_train).reveal().argmax(axis=1)
matches = predicted_classes.unwrap() == correct_classes.unwrap()
print("Training accuracy: ", sum(matches)/len(matches))

predicted_classes = classifier.predict(NativeTensor(x_test_features)).reveal().argmax(axis=1)
correct_classes   = NativeTensor(y_test).reveal().argmax(axis=1)
matches = predicted_classes.unwrap() == correct_classes.unwrap()
print("Test accuracy: ", sum(matches)/len(matches))

Elapsed:  0:00:34.537999
Training accuracy:  0.807933333333
Test accuracy:  0.811767126106


### ... using PublicEncodedTensor

In [44]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    PublicEncodedTensor(x_train_features), 
    PublicEncodedTensor(y_train), 
    loss=CrossEntropy(), 
    epochs=3,
    verbose=True
)
stop = datetime.now()
print("Elapsed: ", stop - start)

predicted_classes = classifier.predict(PublicEncodedTensor(x_train_features)).reveal().argmax(axis=1)
correct_classes   = PublicEncodedTensor(y_train).reveal().argmax(axis=1)
matches = predicted_classes.unwrap() == correct_classes.unwrap()
print("Training accuracy: ", sum(matches)/len(matches))

predicted_classes = classifier.predict(PublicEncodedTensor(x_test_features)).reveal().argmax(axis=1)
correct_classes   = PublicEncodedTensor(y_test).reveal().argmax(axis=1)
matches = predicted_classes.unwrap() == correct_classes.unwrap()
print("Test accuracy: ", sum(matches)/len(matches))

2017-12-20 23:20:08.734787 Epoch 0
2017-12-20 23:20:08.735008 Batch 0
2017-12-20 23:20:25.667752 Batch 1
2017-12-20 23:20:40.171538 Batch 2
2017-12-20 23:20:54.724925 Batch 3
2017-12-20 23:21:10.256817 Batch 4
2017-12-20 23:21:27.828717 Batch 5
2017-12-20 23:21:41.292288 Batch 6
2017-12-20 23:21:54.493442 Batch 7
2017-12-20 23:22:10.738909 Batch 8
2017-12-20 23:22:24.807991 Batch 9
2017-12-20 23:22:38.101790 Batch 10
2017-12-20 23:22:51.788759 Batch 11
2017-12-20 23:23:05.225547 Batch 12
2017-12-20 23:23:18.713219 Batch 13
2017-12-20 23:23:32.841993 Batch 14
2017-12-20 23:23:48.801327 Batch 15
2017-12-20 23:24:05.509415 Batch 16
2017-12-20 23:24:20.601524 Batch 17
2017-12-20 23:24:35.480640 Batch 18
2017-12-20 23:24:51.045577 Batch 19
2017-12-20 23:25:18.946956 Batch 20
2017-12-20 23:25:33.396184 Batch 21
2017-12-20 23:25:48.107313 Batch 22
2017-12-20 23:26:03.492585 Batch 23
2017-12-20 23:26:18.597171 Batch 24
2017-12-20 23:26:33.777803 Batch 25
2017-12-20 23:26:47.713903 Batch 26
201

2017-12-21 00:15:36.112252 Batch 224
2017-12-21 00:15:49.147885 Batch 225
2017-12-21 00:16:02.275080 Batch 226
2017-12-21 00:16:15.357161 Batch 227
2017-12-21 00:16:28.259776 Batch 228
2017-12-21 00:16:41.312151 Batch 229
2017-12-21 00:16:54.262818 Batch 230
2017-12-21 00:17:07.208967 Batch 231
2017-12-21 00:17:20.186806 Batch 232
2017-12-21 00:17:33.321714 Batch 233
2017-12-21 00:17:46.339455 Batch 234
2017-12-21 00:17:59.857085 Batch 235
2017-12-21 00:18:12.918212 Batch 236
2017-12-21 00:18:26.117490 Batch 237
2017-12-21 00:18:39.183155 Batch 238
2017-12-21 00:18:52.113747 Batch 239
2017-12-21 00:19:05.078271 Batch 240
2017-12-21 00:19:18.061089 Batch 241
2017-12-21 00:19:31.263010 Batch 242
2017-12-21 00:19:44.218307 Batch 243
2017-12-21 00:19:57.447136 Batch 244
2017-12-21 00:20:10.499627 Batch 245
2017-12-21 00:20:23.547281 Batch 246
2017-12-21 00:20:36.600920 Batch 247
2017-12-21 00:20:49.560083 Batch 248
2017-12-21 00:21:02.831996 Batch 249
2017-12-21 00:21:15.863218 Batch 250
2

2017-12-21 01:03:51.707854 Batch 446
2017-12-21 01:04:04.769232 Batch 447
2017-12-21 01:04:17.749098 Batch 448
2017-12-21 01:04:30.804826 Batch 449
2017-12-21 01:04:43.844579 Batch 450
2017-12-21 01:04:56.891440 Batch 451
2017-12-21 01:05:09.896520 Batch 452
2017-12-21 01:05:22.809570 Batch 453
2017-12-21 01:05:36.039189 Batch 454
2017-12-21 01:05:49.243027 Batch 455
2017-12-21 01:06:02.226552 Batch 456
2017-12-21 01:06:15.126174 Batch 457
2017-12-21 01:06:28.326734 Batch 458
2017-12-21 01:06:41.575189 Batch 459
2017-12-21 01:06:54.733157 Batch 460
2017-12-21 01:07:07.886111 Batch 461
2017-12-21 01:07:20.847940 Batch 462
2017-12-21 01:07:34.069355 Batch 463
2017-12-21 01:07:46.907974 Batch 464
2017-12-21 01:08:00.056345 Batch 465
2017-12-21 01:08:13.009679 Batch 466
2017-12-21 01:08:26.047670 Batch 467
2017-12-21 01:08:39.098959 Batch 468
2017-12-21 01:08:49.392874 Epoch 1
2017-12-21 01:08:49.395237 Batch 0
2017-12-21 01:09:02.493018 Batch 1
2017-12-21 01:09:15.659413 Batch 2
2017-12-2

2017-12-21 01:52:29.541677 Batch 201
2017-12-21 01:52:42.369852 Batch 202
2017-12-21 01:52:55.482456 Batch 203
2017-12-21 01:53:08.402221 Batch 204
2017-12-21 01:53:21.368864 Batch 205
2017-12-21 01:53:34.408584 Batch 206
2017-12-21 01:53:47.362138 Batch 207
2017-12-21 01:54:00.340580 Batch 208
2017-12-21 01:54:13.344146 Batch 209
2017-12-21 01:54:26.297810 Batch 210
2017-12-21 01:54:39.347039 Batch 211
2017-12-21 01:54:52.515168 Batch 212
2017-12-21 01:55:05.570197 Batch 213
2017-12-21 01:55:18.908684 Batch 214
2017-12-21 01:55:31.864852 Batch 215
2017-12-21 01:55:44.829500 Batch 216
2017-12-21 01:55:57.688489 Batch 217
2017-12-21 01:56:10.723364 Batch 218
2017-12-21 01:56:23.740225 Batch 219
2017-12-21 01:56:36.746153 Batch 220
2017-12-21 01:56:49.827491 Batch 221
2017-12-21 01:57:02.887534 Batch 222
2017-12-21 01:57:15.731480 Batch 223
2017-12-21 01:57:28.822001 Batch 224
2017-12-21 01:57:41.944292 Batch 225
2017-12-21 01:57:54.811995 Batch 226
2017-12-21 01:58:07.711958 Batch 227
2

2017-12-21 02:40:38.730184 Batch 423
2017-12-21 02:40:51.770408 Batch 424
2017-12-21 02:41:04.786619 Batch 425
2017-12-21 02:41:17.800220 Batch 426
2017-12-21 02:41:30.867140 Batch 427
2017-12-21 02:41:44.016291 Batch 428
2017-12-21 02:41:57.124508 Batch 429
2017-12-21 02:42:10.127382 Batch 430
2017-12-21 02:42:23.121088 Batch 431
2017-12-21 02:42:36.303123 Batch 432
2017-12-21 02:42:49.406668 Batch 433
2017-12-21 02:43:03.329723 Batch 434
2017-12-21 02:43:16.189634 Batch 435
2017-12-21 02:43:29.453648 Batch 436
2017-12-21 02:43:42.476897 Batch 437
2017-12-21 02:43:55.427244 Batch 438
2017-12-21 02:44:08.461184 Batch 439
2017-12-21 02:44:21.542567 Batch 440
2017-12-21 02:44:34.436706 Batch 441
2017-12-21 02:44:47.333359 Batch 442
2017-12-21 02:45:00.381257 Batch 443
2017-12-21 02:45:13.334120 Batch 444
2017-12-21 02:45:26.416495 Batch 445
2017-12-21 02:45:39.466681 Batch 446
2017-12-21 02:45:52.636251 Batch 447
2017-12-21 02:46:05.546108 Batch 448
2017-12-21 02:46:18.626177 Batch 449
2

2017-12-21 03:29:09.214932 Batch 178
2017-12-21 03:29:22.272029 Batch 179
2017-12-21 03:29:35.344571 Batch 180
2017-12-21 03:29:48.222636 Batch 181
2017-12-21 03:30:01.164478 Batch 182
2017-12-21 03:30:14.308805 Batch 183
2017-12-21 03:30:27.220104 Batch 184
2017-12-21 03:30:40.272861 Batch 185
2017-12-21 03:30:53.228404 Batch 186
2017-12-21 03:31:06.286393 Batch 187
2017-12-21 03:31:19.276629 Batch 188
2017-12-21 03:31:32.403957 Batch 189
2017-12-21 03:31:45.522006 Batch 190
2017-12-21 03:31:58.505529 Batch 191
2017-12-21 03:32:11.472051 Batch 192
2017-12-21 03:32:24.580597 Batch 193
2017-12-21 03:32:37.706569 Batch 194
2017-12-21 03:32:50.806842 Batch 195
2017-12-21 03:33:03.868908 Batch 196
2017-12-21 03:33:16.990510 Batch 197
2017-12-21 03:33:29.969401 Batch 198
2017-12-21 03:33:43.064792 Batch 199
2017-12-21 03:33:56.184930 Batch 200
2017-12-21 03:34:09.305195 Batch 201
2017-12-21 03:34:22.259031 Batch 202
2017-12-21 03:34:35.292550 Batch 203
2017-12-21 03:34:48.251475 Batch 204
2

2017-12-21 04:17:17.833233 Batch 400
2017-12-21 04:17:30.844565 Batch 401
2017-12-21 04:17:43.760911 Batch 402
2017-12-21 04:17:56.536044 Batch 403
2017-12-21 04:18:09.450633 Batch 404
2017-12-21 04:18:22.346001 Batch 405
2017-12-21 04:18:35.322155 Batch 406
2017-12-21 04:18:48.274777 Batch 407
2017-12-21 04:19:01.356249 Batch 408
2017-12-21 04:19:14.431548 Batch 409
2017-12-21 04:19:27.298748 Batch 410
2017-12-21 04:19:40.429092 Batch 411
2017-12-21 04:19:53.437884 Batch 412
2017-12-21 04:20:06.468752 Batch 413
2017-12-21 04:20:19.360363 Batch 414
2017-12-21 04:20:32.467905 Batch 415
2017-12-21 04:20:45.405832 Batch 416
2017-12-21 04:20:58.390357 Batch 417
2017-12-21 04:21:11.637115 Batch 418
2017-12-21 04:21:24.532638 Batch 419
2017-12-21 04:21:37.542089 Batch 420
2017-12-21 04:21:50.531791 Batch 421
2017-12-21 04:22:03.468524 Batch 422
2017-12-21 04:22:16.555974 Batch 423
2017-12-21 04:22:29.561640 Batch 424
2017-12-21 04:22:42.577468 Batch 425
2017-12-21 04:22:55.577113 Batch 426
2

### ... using PrivateEncodedTensor

In [ ]:
classifier.initialize()

start = datetime.now()
classifier.fit(
    PrivateEncodedTensor(x_train_features), 
    PrivateEncodedTensor(y_train), 
    loss=CrossEntropy(), 
    epochs=3,
    verbose=True
)
stop = datetime.now()
print("Elapsed: ", stop - start)

predicted_classes = classifier.predict(PrivateEncodedTensor(x_train_features)).reveal().argmax(axis=1)
correct_classes   = PrivateEncodedTensor(y_train).reveal().argmax(axis=1)
matches = predicted_classes.unwrap() == correct_classes.unwrap()
print("Training accuracy: ", sum(matches)/len(matches))

predicted_classes = classifier.predict(PrivateEncodedTensor(x_test_features)).reveal().argmax(axis=1)
correct_classes   = PrivateEncodedTensor(y_test).reveal().argmax(axis=1)
matches = predicted_classes.unwrap() == correct_classes.unwrap()
print("Test accuracy: ", sum(matches)/len(matches))

2017-12-21 09:36:41.577781 Epoch 0
2017-12-21 09:36:41.588037 Batch 0
2017-12-21 09:37:42.555613 Batch 1
2017-12-21 09:39:01.436805 Batch 2
2017-12-21 09:40:19.389435 Batch 3
2017-12-21 09:41:37.540146 Batch 4
2017-12-21 09:42:55.071089 Batch 5
2017-12-21 09:44:13.351323 Batch 6
2017-12-21 09:45:32.110302 Batch 7
2017-12-21 09:46:50.152837 Batch 8
2017-12-21 09:48:07.643703 Batch 9
2017-12-21 09:49:25.244494 Batch 10
2017-12-21 09:50:43.097700 Batch 11
2017-12-21 09:52:01.186041 Batch 12
2017-12-21 09:53:18.893244 Batch 13
2017-12-21 09:54:37.400129 Batch 14
2017-12-21 09:55:55.756159 Batch 15
2017-12-21 09:57:13.171174 Batch 16
2017-12-21 09:58:30.701955 Batch 17
2017-12-21 09:59:48.061530 Batch 18
2017-12-21 10:01:05.459641 Batch 19
2017-12-21 10:02:23.059789 Batch 20
2017-12-21 10:03:40.940688 Batch 21
2017-12-21 10:04:58.871584 Batch 22
2017-12-21 10:06:16.740553 Batch 23
2017-12-21 10:07:34.843021 Batch 24
2017-12-21 10:08:52.392324 Batch 25
2017-12-21 10:10:09.615485 Batch 26
201

2017-12-21 14:28:45.556544 Batch 224
2017-12-21 14:30:04.862993 Batch 225
2017-12-21 14:31:23.407215 Batch 226
2017-12-21 14:32:40.967725 Batch 227
2017-12-21 14:33:59.223759 Batch 228
2017-12-21 14:35:17.844160 Batch 229
2017-12-21 14:36:35.327639 Batch 230
2017-12-21 14:37:53.195699 Batch 231
2017-12-21 14:39:12.596389 Batch 232
2017-12-21 14:40:30.410309 Batch 233
2017-12-21 14:41:51.149009 Batch 234
2017-12-21 14:43:09.578109 Batch 235
2017-12-21 14:44:46.278563 Batch 236
2017-12-21 14:46:05.691074 Batch 237
2017-12-21 14:47:30.524773 Batch 238
2017-12-21 14:48:48.416248 Batch 239
2017-12-21 14:50:16.476053 Batch 240
2017-12-21 14:51:38.789878 Batch 241
2017-12-21 14:52:59.938647 Batch 242
2017-12-21 14:54:27.917244 Batch 243
2017-12-21 14:55:55.446902 Batch 244
2017-12-21 14:57:19.154487 Batch 245
2017-12-21 14:58:47.207775 Batch 246
2017-12-21 15:00:11.818873 Batch 247
2017-12-21 15:01:41.659920 Batch 248
2017-12-21 15:03:08.009680 Batch 249
2017-12-21 15:04:31.776907 Batch 250
2

2017-12-21 19:31:58.450895 Batch 446
2017-12-21 19:33:15.641005 Batch 447
2017-12-21 19:34:33.497795 Batch 448
2017-12-21 19:35:50.988504 Batch 449
2017-12-21 19:37:08.517889 Batch 450
2017-12-21 19:38:25.211150 Batch 451
2017-12-21 19:39:42.337831 Batch 452
2017-12-21 19:40:59.486249 Batch 453
2017-12-21 19:42:17.003186 Batch 454
2017-12-21 19:43:35.300281 Batch 455
2017-12-21 19:44:52.933574 Batch 456
2017-12-21 19:46:09.910806 Batch 457
2017-12-21 19:47:27.216048 Batch 458
2017-12-21 19:48:44.918332 Batch 459
2017-12-21 19:50:02.385672 Batch 460
2017-12-21 19:51:19.913820 Batch 461
2017-12-21 19:52:36.622039 Batch 462
2017-12-21 19:53:54.888948 Batch 463
2017-12-21 19:55:12.041246 Batch 464
2017-12-21 19:56:29.414009 Batch 465
2017-12-21 19:57:46.641657 Batch 466
2017-12-21 19:59:04.029320 Batch 467
2017-12-21 20:00:21.125044 Batch 468
2017-12-21 20:01:20.955143 Epoch 1
2017-12-21 20:01:20.955350 Batch 0
2017-12-21 20:02:37.794684 Batch 1
2017-12-21 20:03:55.113883 Batch 2
2017-12-2

2017-12-22 22:25:03.326807 Batch 201
2017-12-22 22:26:27.716471 Batch 202
2017-12-22 22:27:56.611758 Batch 203
2017-12-22 22:29:23.817071 Batch 204
2017-12-22 22:30:45.485506 Batch 205
2017-12-22 22:34:35.172226 Batch 206
2017-12-22 22:36:05.314470 Batch 207
2017-12-22 22:37:41.567079 Batch 208
2017-12-22 22:39:09.978503 Batch 209
2017-12-22 22:40:33.595576 Batch 210
2017-12-22 22:41:59.206862 Batch 211
2017-12-22 22:43:27.470553 Batch 212
2017-12-22 22:44:47.755588 Batch 213
2017-12-22 22:46:12.727516 Batch 214
2017-12-22 22:47:35.513348 Batch 215
2017-12-22 22:48:46.168455 Batch 216
2017-12-22 22:50:08.990337 Batch 217
2017-12-22 22:51:29.245937 Batch 218
2017-12-22 22:52:54.643624 Batch 219
2017-12-22 22:55:24.456344 Batch 220
2017-12-22 22:56:58.562186 Batch 221
2017-12-22 22:58:20.613937 Batch 222
2017-12-22 22:59:47.239931 Batch 223
2017-12-22 23:01:15.825605 Batch 224
2017-12-22 23:02:38.742667 Batch 225
2017-12-22 23:04:17.012535 Batch 226
2017-12-22 23:05:43.750373 Batch 227
2

2017-12-23 16:01:11.545709 Batch 423
2017-12-23 16:02:29.618244 Batch 424
2017-12-23 16:03:47.707330 Batch 425
2017-12-23 16:05:06.145007 Batch 426
2017-12-23 16:06:24.606452 Batch 427
2017-12-23 16:07:41.904813 Batch 428
2017-12-23 16:09:00.216760 Batch 429
2017-12-23 16:10:17.816315 Batch 430
2017-12-23 16:11:35.762109 Batch 431
2017-12-23 16:12:54.516634 Batch 432
2017-12-23 16:14:12.228619 Batch 433
2017-12-23 16:15:29.958203 Batch 434
2017-12-23 16:16:46.751859 Batch 435
2017-12-23 16:18:05.025286 Batch 436
2017-12-23 16:19:22.200805 Batch 437
2017-12-23 16:20:39.874525 Batch 438
2017-12-23 16:21:59.524864 Batch 439
2017-12-23 16:23:20.021591 Batch 440
2017-12-23 16:24:37.924087 Batch 441
2017-12-23 16:25:55.604673 Batch 442
2017-12-23 16:27:13.402117 Batch 443
2017-12-23 16:28:36.050994 Batch 444
2017-12-23 16:29:55.363923 Batch 445
2017-12-23 16:31:12.862068 Batch 446
2017-12-23 16:32:31.116981 Batch 447
2017-12-23 16:33:48.598106 Batch 448
2017-12-23 16:35:06.830588 Batch 449
2

2017-12-23 20:52:59.399874 Batch 178
2017-12-23 20:54:18.287399 Batch 179
2017-12-23 20:55:37.133560 Batch 180
2017-12-23 20:56:59.492244 Batch 181
2017-12-23 20:58:21.198011 Batch 182
2017-12-23 20:59:39.728720 Batch 183
2017-12-23 21:00:59.771475 Batch 184
2017-12-23 21:02:17.593646 Batch 185
2017-12-23 21:03:35.013098 Batch 186
2017-12-23 21:04:53.176940 Batch 187
2017-12-23 21:06:10.834926 Batch 188
2017-12-23 21:07:28.479328 Batch 189
2017-12-23 21:08:45.885640 Batch 190
2017-12-23 21:10:04.203447 Batch 191
2017-12-23 21:11:22.085943 Batch 192
2017-12-23 21:12:42.554551 Batch 193
2017-12-23 21:14:00.927333 Batch 194
2017-12-23 21:15:19.416865 Batch 195
2017-12-23 21:16:38.387975 Batch 196
2017-12-23 21:17:59.618625 Batch 197
2017-12-23 21:19:17.472705 Batch 198
2017-12-23 21:20:34.763870 Batch 199
2017-12-23 21:21:51.746019 Batch 200
2017-12-23 21:23:09.327841 Batch 201
2017-12-23 21:24:28.028294 Batch 202
2017-12-23 21:25:45.276470 Batch 203
2017-12-23 21:27:02.728111 Batch 204
2

2017-12-24 15:50:25.468176 Batch 400
2017-12-24 15:51:46.687640 Batch 401
2017-12-24 15:53:07.311086 Batch 402
2017-12-24 15:54:28.212604 Batch 403
2017-12-24 15:55:51.004881 Batch 404
2017-12-24 15:57:25.010250 Batch 405
2017-12-24 15:58:48.387362 Batch 406
2017-12-24 16:00:11.612007 Batch 407
2017-12-24 16:01:42.516277 Batch 408
2017-12-24 16:03:12.744907 Batch 409
2017-12-24 16:04:33.545778 Batch 410
2017-12-24 16:05:51.597255 Batch 411
2017-12-24 16:07:09.757118 Batch 412
2017-12-24 16:08:27.362532 Batch 413
2017-12-24 16:09:43.126227 Batch 414
2017-12-24 16:10:58.409474 Batch 415
2017-12-24 16:12:15.524126 Batch 416
2017-12-24 16:13:31.259743 Batch 417
2017-12-24 16:14:46.928425 Batch 418
2017-12-24 16:16:02.223747 Batch 419
2017-12-24 16:17:18.488021 Batch 420
2017-12-24 16:18:33.797513 Batch 421
2017-12-24 16:19:49.955785 Batch 422
2017-12-24 16:21:24.245838 Batch 423
2017-12-24 16:22:48.201546 Batch 424
2017-12-24 16:24:05.829882 Batch 425
2017-12-24 16:25:27.244507 Batch 426
2

In [1]:
classifier.predict()

NameError: name 'classifier' is not defined